# Django Tutorial
[See Django 2.2 Documentation Contents]( https://docs.djangoproject.com/en/2.2/contents/)

## Install Django
### Python Virtual Environment 구축
#### PyCharm IDE에서
Create New Project:
> Project Interpreter를 `Virtualenv`를 선택하고 Base interpreter를 선택한다.
 (이전에 만든 virtualenv도 필요하면 선택할 수 있다. 이것을 재사용함)

Source를 GitHub 등에서 clone해서 가져왔을 때, Open:
> File >> Settings >> Poject >> Project interpreter: 설정 아이콘을 클릭하고 `Add...`를 선택하여 Virtual Environment를 설정한다. (Create New Project에서와 동일)

#### Command line에서

Current directory에 Python 가상환경이 설치될 directory 명을 `myenv`라 하자.
`venv`라는 가상환경 생성해 주는 module(Python3.4 이상에서 포함됨)을 실행시키자.  
```
> python -m venv myenv
```
그러면, myenv directory 하부에 새로운 python.exe가 설치되고, site-packeges/ 밑에 pip와 setuptool만 설치된다. 

가상환경을 활성화한다. (사실은 환경변수 PATH와 PYTHONPATH를 venv를 사용하도록 조정하는 것이다.)

Windows:
```
> myenv\Scripts\activate.bat
(myenv) > 
```
Linux:
```
$ source myenv/bin/activate
(myenv) $
```
(myenv) 프롬프트가 생기면, 이제부터 virtual environenment이 적용된다. 즉, python version도 지정되었고, 추가 설치되는 package들도 이 환경에서만 독립적으로 설치되고 영향을 준다. 다시 말해, 이전에 사용한 시스템 python 인터프리터와 package에 영향을 주고 받지 않는 나만의 고유 환경을 구성할 수 있다. 

사용자 code와 package간, 또는 package간에 dependency가 있는 경우에 유용하며, 다른 기계로 porting하기 편리하다.

가상환경을 비활성화 시키려면(끝내려면),

Windows:
```
(myenv) > deactivate.bat
>
```
Linux:
```
(myenv) $ deactivate
$
```

Python을 실행시켜 `sys.path`를 확인해 보자. Module을 찾을 path를 확인해 보자.

### Virtual env에서 Django package 설치하기

#### PyCharm IDE에서
> File >> Settings >> Poject >> Project interpreter: site-packages directory에 설치된 pip와 setup tools가 list되어 있다. 이제 새 package를 설치하려면, `+`를 누르고
`django`를 검색해 보자. `Django`를 선택해서 `Install Package`를 클릭. 
특정 version을 선택할 수도 있다. -->` Specify version` 칸에 version 번호를 입력

#### Command line에서
```
> pip install Django
```

Django 1.11.20 version을 설치하려면,
```
> pip install Django==1.11.20
```

Note: 이 가상환경에서 설치된 package들은 기존에 설치된 Python interpreter에 영향을 주지 않는다.

## Setup a database
PostgreSQL, MySQL, or Oracle 등 대형 DBMS:
- 별도 설치가 필요하다. 
- 이 DBMS는 application server와 별도의 machine에서 실행 가능하며 TCP socket 통신으로 query가 전달된다. 
- 물론, 사용권한이 있어야 한다.
- Python interface를 위한 DB API 설치 필요

>참고: Python MySQL https://www.w3schools.com/python/python_mysql_getstarted.asp

SQLite3:
- Django의 default DB이며 표준 Python에 포함되어 있다. 
- 다만, SQL을 사용하지 않고 DB를 access하려면, [SQLite Database Browser](https://sqlitebrowser.org/)를 설치하자.
- 참고: [SQLite로 가볍게 배우는 데이터베이스](https://wikidocs.net/book/1530) 

## [Django at a glance](https://docs.djangoproject.com/en/2.2/intro/overview/)

Django Architecture (MVTC)
![Django Architecture](django_arch.jpg)

Django Layout
![Django Layout](layout.png)

## [Writing your first Django app, part 1](https://docs.djangoproject.com/en/2.2/intro/tutorial01/)
###### Creating a project
```
$ django-admin startproject mysite
```

###### The development server
```
$ python manage.py runserver
```

###### Creating the Polls app
```
$ python manage.py startapp polls
```

###### Write your first view
- polls/views.py
- polls/urls.py
- mysite/urls.py

URLconfs
```
path(route, view, kwargs=None, name=None)
re_path(route, view, kwargs=None, name=None)
```

###### Run
Run server:
```
$ python manage.py runserver
```

Go to http://localhost:8000/polls/ in your browser


## [Writing your first Django app, part 2](https://docs.djangoproject.com/en/2.2/intro/tutorial02/)

설정 - `mysite/settings.py` 파일을 다음과 같이 수정한다. (comment 참조)
```Python
INSTALLED_APPS = [
    # Insert your app here
    # 'polls.apps.PollsConfig',
    'django.contrib.admin',
    'django.contrib.auth',
    ...
    
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        # 'DIRS': [],
        'DIRS': [os.path.join(BASE_DIR, 'templates')],  # project's templates
        'APP_DIRS': True,               # lookup app's templates dir
        ...

# TIME_ZONE = 'UTC'
TIME_ZONE = 'Asia/Seoul'
```

###### Database setup
Django가 만들어 놓은 admin, auth 등 application들이 사용하는 DB schema가 미리 정의되어 있다. 이를 DB에 이동시킨다. 즉, db.sqlite3 DB에 table을 creation한다.
```
python manage.py migrate
```

###### Creating models
`polls/models`.py 파일를 작성한다. DB schema는 Python class로 대응된다. 

###### Activating models
`mysite/settings.py`에 `'polls.apps.PollsConfig'`을 추가하여 polls app을 등록한다.

Question, Choice table을 DB에 create시킬 `Migration` class를 생성한다.
```
$ python manage.py makemigrations polls
```

`polls/migrations/0001_initial.py`가 생성되었다. 여기에는 DB table을 create하기 위한 specification이 기술되어 있다.

>참고: 만든 migration을 가지고 DB에 table을 creationg하기 위해 어떠한 SQL statements 생성되는지 확인해 보자. 

>`$ python manage.py sqlmigrate polls 0001`


작성한 model(즉, Python class)에서 DB에 table을 생성하자. 변경된 migration들이 존재하면(즉, model에 변경이 있으면) DB에 이를 적용한다. (Table field에 변경이 있거나, 삭제 등)
```
$ python manage.py migrate
```

이제 class Question과 Choice는 DB에 polls_question, polls_choice라는 table명으로 생성되었다. 

*SQLite DB Browser*로 확인해 보자.

> `makemigrations [app_name]` 
- generates the sql command to create the table corresponding to each class you made in models.py file. 
- 모델이 변경됐을 때만, migrations/ 에 새로운 .py 파일이 생긴다.

> `migrate [app_name]`
- will create the table in database using the commands which have been generated by makemigrations.

> `sqlmigrate`:
- shows all the SQL commands which has been generated by makemigrations.

> `showmigrations`:
- show what migrations exist in a Django project

###### Playing with the API
Python shell로 interactive하게 DB에 insert, select할 수 있지만, 장고가 제공하는 database API를 이해하도록 하자. 실제로는 admin application에서 DB를 handling할 수 있으니 DB에 data 삽입은 미루기로 한다.

`polls/models.py`는 tutorial에 따라 수정한다.

###### Introducing the Django Admin
1. Creating an admin user
```
$ python manage.py createsuperuser
```
여러분의 user id와 e-mail, password를 등록한다.
2. Start the development server <br>
Development server를 run시키고 http://localhost:8000/admin 로 가서 loggin하자. 
3. Enter the admin site
4. Make the poll app modifiable in the admin
5. Explore the free admin functionality

###### 참고: Admin이 DB에 설문 입력하기
Admin이 browser에서 설문 데이터를 입력할 수 있게 다음 code로 대체해 보자. 

polls/admin.py:
```Python
from django.contrib import admin

from .models import Question, Choice

class ChoiceInline(admin.TabularInline):
    model = Choice
    extra = 3

class QuestionAdmin(admin.ModelAdmin):
    list_display = ('question_text', 'pub_date')
    list_filter = ['pub_date']
    inlines = [ChoiceInline]

admin.site.register(Question, QuestionAdmin)
```
Browser에서 몇 개의 설문을 입력하자.

## [Writing your first Django app, part 3](https://docs.djangoproject.com/en/2.2/intro/tutorial03/)
Overview

Writing more views

Write views that actually do something

Raising a 404 error

Use the template system

Removing hardcoded URLs in templates

Namespacing URL names

## [Writing your first Django app, part 4](https://docs.djangoproject.com/en/2.2/intro/tutorial04/)
Write a simple form

Use generic views: Less code is better

## [Writing your first Django app, part 5](https://docs.djangoproject.com/en/2.2/intro/tutorial05/)
Introducing automated testing

Basic testing strategies

Writing our first test

Test a view

When testing, more is better

Further testing

What’s next?

## [Writing your first Django app, part 6](https://docs.djangoproject.com/en/2.2/intro/tutorial06/)
Customize your app’s look and feel

Adding a background-image

## [Writing your first Django app, part 7](https://docs.djangoproject.com/en/2.2/intro/tutorial07/)

Customize the admin form

Adding related objects

Customize the admin change list

Customize the admin look and feel

Customize the admin index page

What’s next?

## Production and Deployment
### Use GitHub
Source code는 GitHub를 이용하면 이전이 가능하다. 단, database는 버전관리 대상이면 안된다. (.gitignore에 `*.sqlites3` 라인을 추가하자.)

### Database 이전
Database의 content는 machine과 OS에 따라 data가 다르게 표현될 수 있기 때문에 그대로 이식할 수 없다. 따라서 이식성이 있는 JSON으로 저장했다가 server에 다시 loading해야 한다.

Dump:
```
python ./manage.py dumpdata --exclude auth.permission --exclude contenttypes > db.json
```
> 인증과 contenttypes는 dump에서 제외해야한다.

Load:
```
python ./manage.py loaddata db.json
```

### Virtual Environment 재설정 
